In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import pytz
import math
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path1="/content/drive/MyDrive/Risk Project/Data/Technicals"
path2="/content/drive/MyDrive/Risk Project/Data/Fundamentals"

In [ ]:
dir_list = os.listdir(path1)

In [ ]:
for file in dir_list:

  ##calculate technical indicators
  data = pd.read_csv(path1+"/" + file,index_col = 'Datetime')
  data['date'] = pd.to_datetime(data.index).normalize()
  val = list(data.groupby('date').size())

  ##Factor 1: high frequency skewness
  data1 = data.copy()
  data1['r'] = np.log(data['Close']/data['Close'].shift())  
  # r_t,i  = p_t,i/N - p_t,(i-1)/N , p is natural log of the price, N is the num of return observations in a trading day
  data1.loc[data1.index.str.contains("09:30:00"),'r'] = 0
  data1['r^2'] =data1['r']**2
  data1['r^3'] = data1['r']**3
  RDVar_t = data1.groupby(by=["date"])['r^2'].sum()
  cubed_sum = data1.groupby(by=["date"])['r^3'].sum()
  num_root= [math.sqrt(num)for num in val]
  sr = pd.DataFrame(index = data1.date.unique())
  sr['num'] = num_root
  RDSkew = pd.DataFrame(index = data1.date.unique())
  RDSkew['RDSkew'] = (cubed_sum / (RDVar_t ** (1.5))).multiply(sr['num'])
  RDSkew = RDSkew.rolling(window = 5,min_periods=1).mean()
  
  ##Factor 2: Downside Volatility Ratio
  data2 = data1.copy()
  data2['I'] = (data2['r'].apply(lambda x: '1' if x < 0 else '0'))
  data2['I'] = data2['I'].astype(str).astype(float)
  data2 ['r^2 * I'] = data2['r^2'] * data2['I']
  r_I = data2.groupby(by=["date"])['r^2 * I'].sum()
  DownVolratio = pd.DataFrame(index = data2.date.unique())
  DownVolratio['down_vol_ratio'] = (r_I / RDVar_t).multiply(sr['num'])
  DownVolratio = DownVolratio.rolling(window = 5, min_periods=1).mean()

  ##Factor 3: Late Transaction Ratio
  data3 = data.copy()
  vol = data3[['Volume','date']]
  vol.index = pd.to_datetime(vol.index)
  vol['vol_sum'] = vol.Volume.resample('30T').sum()
  vol = vol.dropna(axis= 0)
  vol = vol.drop(['Volume'],axis = 1)
  vol_copy = vol.copy()
  vol_copy.index = vol_copy.index.strftime('%Y-%m-%d %H:%M:%S')
  time = vol_copy.loc[vol_copy.index.str.contains("15:30:00")]
  time.set_index(time.loc[:,"date"],inplace=True)
  time = time.drop(columns =['date'])
  vol_day = pd.DataFrame(index = data2.date.unique())
  vol_day['vol_day'] = data3.groupby(by=["date"])['Volume'].sum()
  last_vol_ratio = pd.merge(time, vol_day, left_index=True, right_index=True)
  last_vol_ratio['late_trans_ratio'] = last_vol_ratio['vol_sum']/last_vol_ratio['vol_day']
  last_vol_ratio = last_vol_ratio.rolling(window = 5,min_periods=1).mean()
  last_vol_ratio = last_vol_ratio.drop(columns =['vol_sum','vol_day'])

  ##Factor 4: Volumn -Price Correlation
  data4 = data.copy()
  vol_day = pd.DataFrame(index = data4.date.unique())
  vol_day['vol_day'] = data4.groupby(by=["date"])['Volume'].sum()
  vol_day = vol_day.loc[vol_day.index.repeat(val)]
  vol_day = vol_day.reset_index()
  vol_day = vol_day.rename(columns={"index": "date"})
  data4['vol_day'] = vol_day["vol_day"].values
  data4['ratio'] = data4['Volume'] /data4['vol_day'] 
  vol_pri_corr = pd.DataFrame(index = data4.date.unique())
  vol_pri_corr['vol_price_corr'] = data4.groupby('date')[['Close','ratio']].corr().unstack().iloc[:,1]
  vol_pri_corr = vol_pri_corr.rolling(window = 5,min_periods=1).mean()

  ##Factor 5: Avg Transaction Outflow Ratio
  data5 = data2.copy()
  data5['Amt'] = data5['VWAP'] * data5['Volume']
  Amt_day = data5.groupby(by=["date"])['Amt'].sum()
  data_outflow = data5.query('I == 1')
  Amt_outflow = data_outflow.groupby(by=["date"])['Amt'].sum()
  Avg_trans_outflow = pd.DataFrame(index = data5.date.unique())
  Avg_trans_outflow['avg_trans_outflow_ratio'] = Amt_outflow / Amt_day
  Avg_trans_outflow = Avg_trans_outflow.rolling(window = 5,min_periods=1).mean()

  ##Factor 6 : Large Order Drives Gain
  data6 = data5.copy()
  rank =data6.groupby('date')['Amt'].rank("dense",ascending = False)
  rank = rank.to_list()
  size = data6.groupby(['date']).size()*0.3
  size = size.repeat(val)
  size = pd.DataFrame(rank -size, columns=['num'])
  size.index = data6.index
  data6 =data6.join (size)
  data6 =data6.loc[data6.num <0]
  data6['r*I+1'] =data6.r *data6.I +1
  large_order_gain = pd.DataFrame(index = data6.date.unique())
  large_order_gain['large_order_drives_gain'] = data6.groupby(by=["date"])['r*I+1'].prod()
  large_order_gain = (large_order_gain['large_order_drives_gain']).rolling(window=5,min_periods=1).agg(lambda x : x.prod()) -1

  ##Technical Summary
  dfs = [RDSkew,DownVolratio,last_vol_ratio,vol_pri_corr,Avg_trans_outflow,large_order_gain]
  indicators = pd.concat(dfs, axis=1)
  indicators.index.names = ['date']
  idx = indicators.index.astype(str).tolist()
  indicators = indicators.reset_index()

  ##calculate fundamental indicators
  df = pd.read_csv(path2+"/" + file)
  df = df.rename(columns ={"Unnamed: 0" : "date"})
  df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')
  df = df.loc[(df.date <'2022-04-01') &(df.date >'2020-07-01')]
  df = df.set_index('date')
  df = df[['netDebt','totalAssets','netIncome','totalStockholderEquity','totalRevenue','commonStockSharesOutstanding','dividendsPaid',
        'totalCashFromOperatingActivities']]
  df = df.fillna(value = 0)

  fundamental = pd.DataFrame(index = df.index)
  fundamental['DR'] = df['netDebt']/df['totalAssets'] ## debt ratio
  fundamental['ROE'] = df['netIncome']/df['totalStockholderEquity'] ## return on equity
  fundamental['PM'] = df['netIncome']/df['totalRevenue'] ## profit margin ratio
  fundamental['deltaR'] = df['totalRevenue'].pct_change() ## % change of revenue growth
  fundamental['deltaCSO'] = df['commonStockSharesOutstanding'].pct_change() ## % change of common stock outstanding
  fundamental['deltaNT'] = df['netIncome'].pct_change() ## change of net income
  fundamental['PR'] = df['dividendsPaid']/df['netIncome'] ## payout ratio
  fundamental['deltaCFO'] = df['totalCashFromOperatingActivities'].pct_change() ## change of cash flow from operating
  fundamental = fundamental.dropna()
  fundamental.index = fundamental.index.to_period('Q')
  fundamental = fundamental.resample('D').ffill()
  fundamental = fundamental.loc[(fundamental.index >'2021-01-01') & (fundamental.index <'2022-03-01')]

  temp=pd.DataFrame()
  for i in idx:
    temp = temp.append(fundamental[fundamental.index == i])
  temp = temp.reset_index()

  ## Indicators Summary
  indicators = pd.merge(indicators,temp,left_index=True, right_index=True)
  indicators = indicators.drop(['date_x','date_y'],axis =1)
  indicators['date'] = temp['date']
  indicators = indicators.set_index('date')
  
  indicators.to_csv("/content/drive/MyDrive/Risk Project/result/" + file)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab